In [0]:

from aadatapipelinecore.core.urn import Urn
import psycopg2
from pyspark.sql import Row
from dateutil.relativedelta import relativedelta
spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")
from aaplproxy.connection import ClusterConnection
from conf import settings
from aadatapipelinecore.core.loader.plproxy import build_db_settings
import datetime as d
import datetime
from datetime import timedelta


def get_daily_date_list():
    result = []
    end = datetime.date(2019, 10, 31)
    start = datetime.date(2016, 12, 01)
    while start <= end:
        month_data_raw = datetime.datetime.strftime(start, '%Y-%m-%d')
        result.append(Row(month_data_raw))
        start += relativedelta(days=1)
    return result


def get_path_date_list(granularity):
    date_list = {}
    if granularity == 'daily':
        collect_date = get_daily_date_list()
    for x in collect_date:
        if date_list.has_key(x[0][:7]):
            date_list[x[0][:7]].append(x[0])
        else:
            date_list[x[0][:7]] = [x[0]]
    date_list = sorted(date_list.items(), key=lambda x: datetime.datetime.strptime(x[0] + str(-01), '%Y-%m-%d'),
                       reverse=False)
    return date_list


PG_AA_HOSTS = [('10.2.10.254', 5432)]
PG_AA_NAME = 'aa_store_db'
PG_AA_ACCESS_ID = 'citus_bdp_prod_app_int_qa'
PG_AA_SECRET_KEY = 'wZw8cfBuuklIskVG'

aa_dsn = (
    "dbname='{db}' user='{user}' password='{password}' "
    "host='{host}' port='{port}'".format(
        db=PG_AA_NAME,
        user=PG_AA_ACCESS_ID,
        host=PG_AA_HOSTS[0][0],
        password=PG_AA_SECRET_KEY,
        port=PG_AA_HOSTS[0][1]
    )
)

urn = Urn(
    namespace="app-qa.db-check.v1",
    owner="app_qa"
)
sql = """select * from store.usage_basic_kpi_fact_v1_p_201701 where date='2017-01-01' limit 50;"""


def query(dsn, sql):
    with psycopg2.connect(dsn) as conn:
        conn.autocommit = True
        with conn.cursor() as cur:
            cur.execute(sql)
            result = cur.fetchall()
            conn.commit()
    return result


def check_v3_db_completeness(date_list, graularity):
    for month in date_list:
        m = month[0][:4] + month[0][5:8]
        for day in month[1]:
            result = query(aa_dsn, sql)
            print result
            break
    
            v3_path = 's3://b2c-prod-data-pipeline-unified-usage/unified/usage.basic-kpi.v5/fact/granularity={}/date={}/'
            v3_count = spark.read.parquet(v3_path.format(graularity, day)).count()
    
            if db_count != v3_count:
                print "Completeness Test Fail!!!! date: {}, v3: {}, db: {}".format(day, v3_count, db_count)
            else:
                print "Completeness Test Pass! date : {}".format(day)
        break


graularity_list = ["daily"]
for graularity in graularity_list:
    check_v3_db_completeness(get_path_date_list(graularity), graularity)

In [0]:

from aadatapipelinecore.core.urn import Urn
import psycopg2
from pyspark.sql import Row
from dateutil.relativedelta import relativedelta

spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")
from aaplproxy.connection import ClusterConnection
from conf import settings
from aadatapipelinecore.core.loader.plproxy import build_db_settings
import datetime as d
import datetime
from datetime import timedelta


def get_daily_date_list():
    result = []
    end = datetime.date(2019, 10, 31)
    start = datetime.date(2016, 12, 01)
    while start <= end:
        month_data_raw = datetime.datetime.strftime(start, '%Y-%m-%d')
        result.append(Row(month_data_raw))
        start += relativedelta(days=1)
    return result


def get_path_date_list(granularity):
    date_list = {}
    if granularity == 'daily':
        collect_date = get_daily_date_list()
    for x in collect_date:
        if date_list.has_key(x[0][:7]):
            date_list[x[0][:7]].append(x[0])
        else:
            date_list[x[0][:7]] = [x[0]]
    date_list = sorted(date_list.items(), key=lambda x: datetime.datetime.strptime(x[0] + str(-01), '%Y-%m-%d'),
                       reverse=False)
    return date_list


PG_AA_HOSTS = [('10.2.10.254', 5432)]
PG_AA_NAME = 'aa_store_db'
PG_AA_ACCESS_ID = 'citus_bdp_prod_app_int_qa'
PG_AA_SECRET_KEY = 'wZw8cfBuuklIskVG'

aa_dsn = (
    "dbname='{db}' user='{user}' password='{password}' "
    "host='{host}' port='{port}'".format(
        db=PG_AA_NAME,
        user=PG_AA_ACCESS_ID,
        host=PG_AA_HOSTS[0][0],
        password=PG_AA_SECRET_KEY,
        port=PG_AA_HOSTS[0][1]
    )
)

urn = Urn(
    namespace="app-qa.db-check.v1",
    owner="app_qa"
)
sql = """select COLUMN_NAME from information_schema.COLUMNS where table_name = 'usage_basic_kpi_fact_v1_p_{}';"""


def query(dsn, sql):
    with psycopg2.connect(dsn) as conn:
        conn.autocommit = True
        with conn.cursor() as cur:
            cur.execute(sql)
            result = cur.fetchall()
            conn.commit()
    return result


def check_v3_db_accuracy(date_list, granularity):
    for month in date_list:
        m = month[0][:4] + month[0][5:8]
        for day in month[1]:
            result = query(aa_dsn, sql.format(m))
            db_columns = [x[0] for x in result]

            v3_path = 's3://b2c-prod-data-pipeline-unified-usage/unified/usage.basic-kpi.v5/fact/granularity={}/date={}/'
            v3_df = spark.read.parquet(v3_path.format(granularity, day))
            v3_df = (
                v3_df
                .withColumnRenamed('est_share_of_category_bytes', 'est_share_of_category_mb')
                .withColumnRenamed('est_average_bytes_per_user', 'est_average_mb_per_user')
                .withColumnRenamed('est_average_bytes_per_session', 'est_average_mb_per_session')
                .drop('_identifier')
            )
            v3_columns = v3_df.columns

            if len(set(v3_columns).difference(set(db_columns))) != 0:
                print 'Accuracy Test Fail!!! date: {}'.format(day)
            else:
                print 'Accuracy Test Pass! date: {}'.format(day)


granularity_list = ["daily"]
for granularity in granularity_list:
    check_v3_db_accuracy(get_path_date_list(granularity), granularity)

In [0]:

from bdce.common.utils import update_application_code
update_application_code(spark, role="BDP-PROD-APP-INT-QA", application_name="zidong-application-autopipeline")

# reload dependencies from temp
spark.sparkContext.addPyFile("/tmp/zeppelin_application_code/libs/python/dependencies.zip")
# spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")
import aaplproxy

In [0]:

from aadatapipelinecore.core.urn import Urn
import psycopg2
from pyspark.sql import Row
from dateutil.relativedelta import relativedelta
from aaplproxy.connection import ClusterConnection
from conf import settings
from aadatapipelinecore.core.loader.plproxy import build_db_settings
import datetime as d
import datetime
from datetime import timedelta


def get_daily_date_list():
    result = []
    end = datetime.date(2019, 12, 31)
    start = datetime.date(2018, 4, 01)
    while start <= end:
        month_data_raw = datetime.datetime.strftime(start, '%Y-%m-%d')
        result.append(Row(month_data_raw))
        start += relativedelta(days=1)
    return result


def get_path_date_list(gran):
    date_list = {}
    if gran == 'daily':
        collect_date = get_daily_date_list()
    for x in collect_date:
        if x[0][:7] in date_list:
            date_list[x[0][:7]].append(x[0])
        else:
            date_list[x[0][:7]] = [x[0]]
    date_list = sorted(date_list.items(), key=lambda d: datetime.datetime.strptime(d[0] + str(-1), '%Y-%m-%d'),
                       reverse=False)
    return date_list


PG_AA_HOSTS = [('10.2.6.141', 5432)]
PG_AA_NAME = 'aa_store_db'
PG_AA_ACCESS_ID = 'citus_bdp_prod_app_int_qa'
PG_AA_SECRET_KEY = 'wZw8cfBuuklIskVG'

aa_dsn = (
    "dbname='{db}' user='{user}' password='{password}' "
    "host='{host}' port='{port}'".format(
        db=PG_AA_NAME,
        user=PG_AA_ACCESS_ID,
        host=PG_AA_HOSTS[0][0],
        password=PG_AA_SECRET_KEY,
        port=PG_AA_HOSTS[0][1]
    )
)

urn = Urn(
    namespace="app-qa.db-check.v1",
    owner="app_qa"
)
sql = """select COLUMN_NAME from information_schema.COLUMNS where table_name = 'usage_basic_kpi_fact_v6_p_{}';"""


def query(dsn, sql):
    with psycopg2.connect(dsn) as conn:
        conn.autocommit = True
        with conn.cursor() as cur:
            cur.execute(sql)
            result = cur.fetchall()
            conn.commit()
    return result


def check_v3_db_accuracy(date_list, granularity):
    """
        date_list:
                [(month,[day1,day2,day3])]
        sample:
            [('2015-12', ['2015-12-27', '2015-12-28', '2015-12-29', '2015-12-30', '2015-12-31'])]
    """
    for month in date_list:
        m = month[0][:4] + month[0][5:8]
        for day in month[1]:
            result = query(aa_dsn, sql.format(m))
            db_columns = [x[0] for x in result]
            db_columns.pop(0)
            v3_path = 's3://b2c-prod-data-pipeline-unified-usage/unified/usage.basic-kpi.v3/fact/' \
                      'granularity={}/date={}/'
            v3_df = spark.read.parquet(v3_path.format(granularity, day))
            v3_df = (
                v3_df
                .drop('_identifier')
            )
            v3_columns = v3_df.columns
            if len(set(v3_columns).difference(set(db_columns))) != 0:
                print 'Accuracy Test Fail!!! date: {}'.format(day)
            else:
                print 'Accuracy Test Pass! date: {}'.format(day)


granularity_list = ["daily"]
for granularity in granularity_list:
    check_v3_db_accuracy(get_path_date_list(granularity), granularity)


In [0]:

from bdce.common.utils import update_application_code
update_application_code(spark, role="BDP-PROD-APP-INT-QA", application_name="aa-int-qa-db-check")

spark.sparkContext.addPyFile("/tmp/zeppelin_application_code/libs/python/dependencies.zip")


In [0]:

from pandas.io import sql as sqlio
from pyspark.sql.types import DoubleType
from pyspark.sql.types import LongType
from pyspark.sql import functions
from pyspark.sql.functions import lit, coalesce
import psycopg2


def query_df(dsn, sql):
    conn = psycopg2.connect(dsn)
    df = sqlio.read_sql_query(sql, conn)
    return df
    
device_code_mapping = {
            1: {'1': 'android-phone', '2': 'android-tablet'},
            2: {'1': 'ios-phone', '2': 'ios-tablet'}}
PG_AA_HOSTS = [('10.2.6.141', 5432)]
PG_AA_NAME = 'aa_store_db'
PG_AA_ACCESS_ID = 'citus_bdp_prod_app_int_qa'
PG_AA_SECRET_KEY = 'wZw8cfBuuklIskVG'

aa_dsn = (
    "dbname='{db}' user='{user}' password='{password}' "
    "host='{host}' port='{port}'".format(
        db=PG_AA_NAME,
        user=PG_AA_ACCESS_ID,
        host=PG_AA_HOSTS[0][0],
        password=PG_AA_SECRET_KEY,
        port=PG_AA_HOSTS[0][1]
    )
)
sql = """select * from usage.usage_basic_kpi_fact_v6 where date='2020-06-14' and granularity='daily';"""
result = query_df(aa_dsn, sql)
citus_db_df = spark.createDataFrame(result)
citus_db_df = citus_db_df.drop('date').drop('granularity').drop('_disable_idx_4_query')

routine_df = spark.read.parquet('s3://aardvark-prod-pdx-mdm-to-int/basic_kpi/version=1.0.0/range_type=DAY/date=2020-06-14')
unified_v1 = (
            routine_df
                .withColumn('device_code', functions.UserDefinedFunction(
                lambda x, y: device_code_mapping[x][y])(routine_df['platform'], routine_df['device_type']))
                .withColumnRenamed('country', 'country_code')
                .withColumn('app_id', routine_df['app_id'].cast(LongType()))
                .withColumnRenamed('AU', 'est_average_active_users')
                .withColumnRenamed('AFU', 'est_average_session_per_user')
                .withColumnRenamed('ADU', 'est_average_session_duration')
                .withColumnRenamed('IP', 'est_install_penetration')
                .withColumnRenamed('AAD', 'est_average_active_days')
                .withColumnRenamed('PAD', 'est_percentage_active_days')
                .withColumnRenamed('MBPU', 'est_average_bytes_per_user')
                .withColumnRenamed('ATU', 'est_average_time_per_user')
                .withColumnRenamed('UP', 'est_usage_penetration')
                .withColumnRenamed('OR', 'est_open_rate')
                .withColumnRenamed('MBPS', 'est_average_bytes_per_session')
                .withColumnRenamed('MBWFT', 'est_percent_of_wifi_total')
                .withColumnRenamed('MBS', 'est_mb_per_second')
                .withColumnRenamed('IS', 'est_installs')
                .withColumnRenamed('SOU', 'est_average_active_users_country_share')
                .withColumnRenamed('SOI', 'est_installs_country_share')
                .withColumn('est_share_of_category_time', lit(None).cast(DoubleType()))
                .withColumn('est_share_of_category_session', lit(None).cast(DoubleType()))
                .withColumn('est_share_of_category_bytes', lit(None).cast(DoubleType()))
                .withColumn('est_panel_size', lit(None).cast(DoubleType()))
                .drop('device_type')
                .drop('platform')
        )

unified_v1 = (
            unified_v1
            .drop("est_mb_per_second", "est_share_of_device_time", "est_share_of_device_session",
                  "est_share_of_device_mb", "est_panel_size")
            .withColumn('est_total_time',
                        unified_v1['est_average_active_users'] * unified_v1['est_average_time_per_user'] / 60)
            .withColumn('est_average_time_per_user', coalesce(unified_v1['est_average_time_per_user'],
                                                              unified_v1['est_average_session_duration'] *
                                                              unified_v1['est_average_session_per_user']) * 1000)
            .withColumn('est_average_session_duration', unified_v1['est_average_session_duration'] * 1000)
        )
unified_v1 = (
            unified_v1
            .withColumn('est_average_bytes_per_session', unified_v1['est_average_bytes_per_session'] * 1024 * 1024)
            .withColumn('est_average_bytes_per_user', unified_v1['est_average_bytes_per_user'] * 1024 * 1024)
            .withColumn('est_share_of_category_bytes', unified_v1['est_share_of_category_bytes'] * 1024 * 1024)
            .withColumn('est_install_base',
                        unified_v1['est_install_penetration'] *
                        unified_v1['est_average_active_users'] / unified_v1['est_usage_penetration'])
            .withColumn('est_population',
                        unified_v1['est_average_active_users'] / unified_v1['est_usage_penetration'])
            .withColumnRenamed('est_average_active_users_country_share', 'est_share_of_users')
            .withColumnRenamed('est_installs_country_share', 'est_share_of_installs')
            .withColumn('est_total_sessions',
                        unified_v1['est_average_active_users'] * unified_v1['est_average_session_per_user'])
        )
unified_v1.createOrReplaceTempView("v1_df")
unified_v3 = spark.sql("""
                    select
                        v1_df.*,
                        ww.est_average_active_users as est_average_active_users_worldwide,
                        ww.est_installs as est_installs_worldwide
                    from v1_df left join
                        (select device_code, app_id, est_installs, est_average_active_users
                         from v1_df where country_code ='WW'
                         ) AS ww
                    on (v1_df.app_id=ww.app_id) and (v1_df.device_code = ww.device_code)
                    """).cache()
unified_v3 = unified_v3.na.fill(0)
unified_v3.filter("app_id=1340929016 and country_code='US' and device_code='ios-phone'").show()

diff_count1 = citus_db_df.select(unified_v3.columns).subtract(unified_v3).count()
print diff_count1
diff_count2 = unified_v3.select(citus_db_df.columns).subtract(citus_db_df).count()
print diff_count2
# unified_v3.select(citus_db_df.columns).subtract(citus_db_df).show()

In [0]:
%%sh
PGPASSWORD='wZw8cfBuuklIskVG' psql -h 10.2.6.141  -U citus_bdp_prod_app_int_qa -d aa_store_db -p 5432 << EOF 
set search_path=usage;
select * from usage_basic_kpi_fact_v6 where granularity='daily' and date='2020-06-14' and app_id=20600000000160 and country_code='Android-phone';
EOF

In [0]:
%%sh
